In [ ]:
%pip install llama-index
%pip install llama_index_llms_ollama
%pip install llama-index-llms-openai
%pip install llama-index-llms-lmstudio
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-ollama
%pip install llama-index-graph-stores-neo4j
%pip install llama-index-packs-neo4j-query-engine
%pip install llama-index-tools-neo4j
%pip install python-dotenv
%pip install neo4j

### Ingest Documents which persisted in Neo4j Database

1.0. Prepare Runtime Env

In [ ]:
import sys
sys.path.append("../")
from IPython.display import Markdown, display
from llama_index.core import Settings
from common.llm import LlmConfig
from common.llm import LocalLLM
config = LlmConfig(local_llm=LocalLLM.LM_STUDIO)
logger = config.logger

3. Read Documents

In [ ]:
from common.reader import read_files_from_directory
docs = read_files_from_directory("./pdf_files")

3. Generate Nodes

In [ ]:
from common.nodes import gen_nodes_by_pipeline
nodes = gen_nodes_by_pipeline(config=config, docs=docs)

4. Preparing Storage Context

In [ ]:
from common.storage import neo4j_storage_context
storage_context = neo4j_storage_context(config=config, database='docs')

In [ ]:
from llama_index.core import KnowledgeGraphIndex
index = KnowledgeGraphIndex.from_documents(
    docs,
    storage_context=storage_context,
    max_triplets_per_chunk=2,
)
# query_engine = index.as_query_engine(
#     include_text=False, response_mode="tree_summarize"
# )
# response = query_engine.query("Tell me more about 林斌")

In [ ]:
storage_context.persist()

In [ ]:
query_engine = index.as_query_engine()
q = "林斌是男的还是女的"
# q = "林斌是哪年出生的"
# q = "林斌有本科学历吗"
resp = query_engine.query(q)
print(resp)